In [4]:
# adding wikipedia library to make my self able to get the information from the given page
import wikipedia as wp
import folium  # map rendering library
from sklearn.cluster import KMeans
import matplotlib.colors as colors
import matplotlib.cm as cm
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
import requests  # library to handle requests
from geopy.geocoders import Nominatim
import json  # library to handle JSON files
import numpy as np  # library to handle data in a vectorized manner

import pandas as pd  # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

# Matplotlib and associated plotting modules

# import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
html = wp.page("List of postal codes of Canada: M").html().encode(
    "UTF-8")  # getting the html page
df = pd.read_html(html, header=0)[0]  # selection the table from  the page
oldDf = df[df.Borough != 'Not assigned']  # filtering Borough
df = oldDf.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(
    list).apply(lambda x: ', '.join(x))
#  adding apply(list) to make me able to transfer the group object to a data frame later
#   adding the (,) if there are more than one string by using lambda and join methods
myFinalDF = df.to_frame().reset_index()
# Converting the group object to data frame and reseting the index
coorDataDf = pd.read_csv('https://cocl.us/Geospatial_data')

for index, row in myFinalDF.iterrows():
    if row['Neighborhood'] == 'Not assigned':  # too obvious isn't it ?
        row['Neighborhood'] = row['Borough']

long = coorDataDf['Longitude']
lat = coorDataDf['Latitude']
myFinalDF['Latitude'] = lat
myFinalDF['Longitude'] = long
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
CLIENT_ID = 'BD1FD31FC4MZ1Z4YPMNGZLQJXPC5FTIASU2OSHMSATOVLVYG'  # your Foursquare ID
CLIENT_SECRET = 'D0JXGY2AQRRZV0K5IOOLTITD0B14UPTCSPEQNLFX3FSVWWVP'
VERSION = '20180605'
radius = 500
LIMIT = 50


def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        # make the GET request
        print(url)

        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame(
        [item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return(nearby_venues)


trontoVenues = getNearbyVenues(names=myFinalDF['Neighborhood'],
                               latitudes=myFinalDF['Latitude'],
                               longitudes=myFinalDF['Longitude']
                               )

toronto_onehot = pd.get_dummies(
    trontoVenues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = trontoVenues['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + \
    list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood']
                           == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(
        drop=True).head(num_top_venues))
    print('\n')


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(
    toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = myFinalDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(
    neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' +
                         str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters


Malvern, Rouge
https://api.foursquare.com/v2/venues/explore?&client_id=BD1FD31FC4MZ1Z4YPMNGZLQJXPC5FTIASU2OSHMSATOVLVYG&client_secret=D0JXGY2AQRRZV0K5IOOLTITD0B14UPTCSPEQNLFX3FSVWWVP&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=50
Rouge Hill, Port Union, Highland Creek
https://api.foursquare.com/v2/venues/explore?&client_id=BD1FD31FC4MZ1Z4YPMNGZLQJXPC5FTIASU2OSHMSATOVLVYG&client_secret=D0JXGY2AQRRZV0K5IOOLTITD0B14UPTCSPEQNLFX3FSVWWVP&v=20180605&ll=43.7845351,-79.16049709999999&radius=500&limit=50
Guildwood, Morningside, West Hill
https://api.foursquare.com/v2/venues/explore?&client_id=BD1FD31FC4MZ1Z4YPMNGZLQJXPC5FTIASU2OSHMSATOVLVYG&client_secret=D0JXGY2AQRRZV0K5IOOLTITD0B14UPTCSPEQNLFX3FSVWWVP&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=50
Woburn
https://api.foursquare.com/v2/venues/explore?&client_id=BD1FD31FC4MZ1Z4YPMNGZLQJXPC5FTIASU2OSHMSATOVLVYG&client_secret=D0JXGY2AQRRZV0K5IOOLTITD0B14UPTCSPEQNLFX3FSVWWVP&v=20180605&ll=43.7709921,-79.21691740000